# Food Classification with CNN - Building a Restaurant Recommendation System

This assignment focuses on developing a deep learning-based food classification system using Convolutional Neural Networks (CNNs). You will build a model that can recognize different food categories and use it to return the food preferences of a user.

## Learning Objectives
- Implement CNNs for image classification
- Work with real-world food image datasets
- Build a preference-detector system

## Background: AI-Powered Food Preference Discovery

The system's core idea is simple:

1. Users upload 10 photos of dishes they enjoy
2. Your CNN classifies these images into the 91 categories
3. Based on these categories, the system returns the user's taste profile

Your task is to develop the core computer vision component that will power this detection engine.

You are given a training ("train" folder) and a test ("test" folder) dataset which have ~45k and ~22k samples respectively. For each one of the 91 classes there is a subdirectory containing the images of the respective class.

## Assignment Requirements

### Technical Requirements
- Implement your own pytorch CNN architecture for food image classification
- Use only the provided training dataset split for training
- Train the network from scratch ; No pretrained weights can be used
- Report test-accuracy after every epoch
- Report all hyperparameters of final model
- Use a fixed seed and do not use any CUDA-features that break reproducibility
- Use Pytorch 2.6

### Deliverables
1. Jupyter Notebook with CNN implementation, training code etc.
2. README file
3. Report (max 3 pages)

Submit your report, README and all code files as a single zip file named GROUP_[number]_NC2425_PA. The names and IDs of the group components must be mentioned in the README.
Do not include the dataset in your submission.

### Grading

1. Correct CNN implementation, training runs on the uni DSLab computers according to the README.MD instructions without ANY exceptions on the DSLab machines: 3pt
2. Perfect 1:1 reproducibility on DSLab machines: 1pt
3. Very clear github-repo-style README.MD with instructions for running the code: 1pt
4. Report: 1pt
5. Model test performance on test-set: interpolated from 30-80% test-accuracy: 0-3pt
6. Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt
7. Bonus point: use an LLM (API) to generate short description / profile of preferences of the simulated user

**If there is anything unclear about this assignment please post your question in the Brightspace discussions forum or send an email**


# Loading the datasets
The dataset is already split into a train and test set in the directories "train" and "test". 

In [1]:
### HYPER-PARAMETER SET-UP ###

# Randomization
random_seed = 42
# Model training
workers = 4
batch_size = 16
num_epochs = 10
# Optimizer (Adam)
learning_rate = 0.001
weight_decay = 0.0
betas = (0.9, 0.99)

In [2]:
### IMPORTS AND DEVICE SELECTION ###

import torch
import numpy as np
import torch.nn as nn
from time import time
from datetime import datetime as dt
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


# Use the GPU to speed up the training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
### DATA LOADING ###

# Allow repeatability
np.random.seed(random_seed)
torch.manual_seed(random_seed) 
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

# Define the transformer for the data
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # not all images are exactly 256x256
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

# Automatically download the dataset and associate folder names as labels
train_dataset = datasets.ImageFolder(root='./train', transform=transform)  # No validation for now
test_dataset = datasets.ImageFolder(root='./test', transform=transform)

# Set up the data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False, num_workers=workers, pin_memory=True)

# Make things easier to understand and utilize
class_names = test_dataset.classes
class_to_idx = test_dataset.class_to_idx
num_classes = len(class_names)
print("Class names:", class_names)
print("Class to index mapping:", class_to_idx)


Class names: ['beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_soup', 'mussels', 'nachos', 'omelette', 'onion_rings', 'oysters', 

# CNN Implementation

In [7]:
### NETWORK ARCHITECHTURE ####

class FoodCNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Start shape: 256x256x3
        self.conv1 = nn.Conv2d(3, 32, kernel_size=6, stride=2, padding=0)
        # After Conv1: 126x126x32
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=3)
        # After Pool1: 42x42x32
        self.conv2 = nn.Conv2d(32, 96, kernel_size=4, stride=2, padding=0)
        # After Conv2: 20x20x96
        self.pool2 = nn.MaxPool2d(kernel_size=6, stride=2)
        # After Pool2: 8x8x96
        self.conv3 = nn.Conv2d(96, 288, kernel_size=3, stride=1, padding=1)
        # After Conv3: 8x8x288
        self.conv4 = nn.Conv2d(288, 96, kernel_size=3, stride=1, padding=1)
        # After Conv4: 8x8x96
        self.pool3 = nn.MaxPool2d(kernel_size=3, stride=1)
        # After Pool3: 6x6x96

        # Flatten to: 3456 - Vector
        self.fc6 = nn.Linear(96*6*6, 1536)
        # After FCL1: 1536 - Vector
        self.fc7 = nn.Linear(1536, 1536)
        # After FCL2: 1536 - Vetor
        self.fc8 = nn.Linear(1536, len(class_names))
        # After FCL3: len(class_names) - Vector

    def forward(self, x):
        # Convolution
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool3(F.relu(self.conv4(x)))
        # Linearization
        x = torch.flatten(x, 1)
        # Fully connected layers
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = self.fc8(x)
       
        return x


# Training the model
Implement your training process below. Report the test-accuracy after every epoch for the training run of the final model.

Hint: before training your model make sure to reset the seed in the training cell, as otherwise the seed may have changed due to previous training runs in the notebook

Note: If you implement automatic hyperparameter tuning, split the train set into train and validation subsets for the objective function.

In [5]:
### HELPER FUNCTIONS ###

def calculate_test_accuracy(model):
    """Calculate the accuracy of a provided model using the global test_loader"""
    # Evaulate using testing
    correct = 0
    model.eval()
    # Don't calculate gradients to speed-up computation
    with torch.no_grad():
        for total, (images, labels) in enumerate(test_loader, start=1):
            # Move the data to the computation device
            images = images.to(device)
            labels = labels.to(device)
            
            # Predict using the model
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            # Safe keep the results to return as a percentage
            correct += (predicted == labels).sum().item()
    return (correct/total) * 100


def train(model, loss_fn, optimizer):
    """Trains a model using the global train_loader"""
    # model.train(True)
    for images, labels in train_loader:
        # Move the data to the computation device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss

In [8]:
### TRAINING BLOCK ###

# Allow repeatability
np.random.seed(random_seed)
torch.manual_seed(random_seed) 
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

# Create model object and connect it to the corresponding processing
model = FoodCNN()
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
model = model.to(device)

# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=betas)

# Begin training, still includes developmental sections
best_vloss = np.inf
for epoch in range(num_epochs):
	# Start epoch
    print(f"Epoch {epoch+1} begin: {dt.now()}.")
    start = time()
    
    # Train the model
    loss = train(model, loss_fn, optimizer)
    print('Completed training after {:.2f} seconds.'.format((time() - start)))
    
    # Output an update
    print('Total epoch time: {:.2f} seconds.'.format((time() - start)))
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print("####################################################################################################")

Epoch 1 begin: 2025-04-30 17:10:39.714730.


KeyboardInterrupt: 

In [ ]:
### MODEL SAVING ###
print("Overwrite? \"Yes\" / else: ")
user_input = input().lower()
if user_input == "yes" or user_input == 'y':
    PATH = './cnn.pth'
    torch.save(model.state_dict(), PATH)
    print("Save complete.")
else:
    PATH = './cnn_backup.pth'
    torch.save(model.state_dict(), PATH)
    print("Original unchanged, Backup overwritten.")

Overwrite? "Yes" / else: 
Original unchanged, Backup overwritten.


# Calculating model performance
Load the best version of your model ( which should be produced and saved by previous cells ), calculate and report the test accuracy.

In [ ]:
### MODEL COMPARISONS ###

# # Load the best model weights
# model2 = FoodCNN().to(device)
# model2.load_state_dict(torch.load("cnn.pth", weights_only=True))

# Calculate and present final scores
final_test_acc = calculate_test_accuracy(model)
print(f"Final Test Accuracy: {final_test_acc:.2f}%")


Final Test Accuracy: 15.13%


# Summary of hyperparameters
Report the hyperparameters ( learning rate etc ) that you used in your final model for reproducibility.

# Simulation of random user
Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt

In [ ]:
# Your code here
# Below an example showing the format of the code output

# Bonus point
Use an LLM (API) to generate a description of the food preference of a user based on 10 images that a potential user could provide. 
Please include an example of the output of your code, especially if you used an API other than the OpenAI API.

This should work well even with differing test images by setting different random seeds for the image selector.